In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("/Users/daniel.al-choboq/Documents/Code/Missions/axa/data/pg15training.csv")
df.head()

In [ ]:
df["Numtppd"].value_counts()

In [ ]:
df['target'] = df['Numtppd'].apply(lambda x: 1 if x != 0 else 0)


In [ ]:
df['target'].value_counts()

In [ ]:
# Convert object columns to categorical
categorical_columns = ['Gender', 'Type', 'Category', 'Occupation', 'SubGroup2', 'Group2']
for col in categorical_columns:
    df[col] = df[col].astype('category')

# Use LightGBM instead of XGBoost
# Add one hot encoder with pipeline scikit learn

# Define features and target
X = df.drop(columns=['Numtppd', 'Numtpbi', 'Indtppd', 'Indtpbi', 'target'])
y = df['target']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the LightGBM model

model = lgb.LGBMClassifier(objective='binary', n_estimators=100, learning_rate=0.1, max_depth=5)

# Train the model
model.fit(X_train, y_train, categorical_feature=categorical_columns)

# Make predictions
y_pred = model.predict(X_test)
 
# Evaluate the model, add more metrics
accuracy = (y_pred == y_test).mean()
print(f"Accuracy: {accuracy}")